<a href="https://colab.research.google.com/github/Domdoug/Regioes-de-Saude/blob/master/Regioes_de_Saude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
nome_colunas = ['CD_MUNICIPIO', 'SG_UF', 'NM_REGIAO_SAUDE_ATUAL', 'NM_MUNICIPIO',
       'NM_REGIAO_SAUDE_ANTERIOR', 'NM_MUNICIPIOS_IN_37', 'ESTADO', 'REGIAO_PAIS']

In [0]:
df_regioes = pd.read_excel('BI_Regionais de saude.xlsx', sheet_name='Dados', skiprows=1, usecols="A:H", names=nome_colunas)

In [0]:
df_regioes.head()

In [0]:
df_regioes.shape

In [0]:
# INICIO DO PROCESSAMENTO

In [0]:
# Esta linha e para agilizar o groupby. Talvez desnecessaria. Lógica do do SAS
df_regioes.sort_values(by=['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'], inplace=True)

In [0]:
# Aplicar na coluna CD_MUNICIPIO uma funcao para colocar os codigos do municipios em formato de conjunto para comparar,
# agrupados por regiao atual e UF, com o agrupamento dos municipios correspondentes da regiao anterior

lista1 = df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'])['CD_MUNICIPIO'].apply(lambda x:list(set(x)))

In [0]:
# Agora a lista para a Região de Saúde anterior
df_regioes.sort_values(by=['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'], inplace=True)

In [0]:
# Agora o código para a lista2: Regiao de Saude Anterior
lista2 = df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'])['CD_MUNICIPIO'].apply(lambda x:list(set(x)))

In [0]:
# Para transformar o formato agrupado em DataFrame
lista1 = lista1.reset_index()
lista2 = lista2.reset_index()

In [0]:
lista1.head()

In [0]:
lista2.head()

In [0]:
lista1.shape

In [0]:
lista2.shape

In [0]:
# ================ TESTES=========================

In [0]:
# verificar se tem regiões que abrangem mais de uma UF:
# df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF']).size()
df_unicos = df_regioes.drop_duplicates(subset=['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'])

In [0]:
df_unicos.shape  #438, 8

In [0]:
# Agrupa para selecionar Região de Saúde no DataFrame. Depois aplica o o metodo duplicated, para selecionar os efetivamente repetidos
linhas_duplicadas = df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'], as_index=False).count()

In [0]:
linhas_duplicadas.shape

In [0]:
# Seleciona todas as linhas duplicadas baseadas em uma lista de colunas. O parametro keep é para exibir todos os repetidos e não o último ou primeiro
linhas_duplicadas = linhas_duplicadas[linhas_duplicadas.duplicated(subset=['NM_REGIAO_SAUDE_ATUAL'], keep=False)]
linhas_duplicadas.shape  #(17, 8)
linhas_duplicadas.head()
# Análise: Observando o campo NM_REGIAO_SAUDE_ATUAL e SG_UF, observa-se que: As regiões repetidas e que são de UF's contíguas São: Baixo Amazonas (AM e PA).
# O restante não são contíguas. Destaque para a Região de Saúde nomeada de "NORTE". Observando-se RJ e ES, que são contíguos, porém as respectivas Regiões situam-se no Norte de cada Estado.


In [0]:
# Até aqui, para a região Norte, não tem Problemas
teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Norte']
# Para o caso da Região Baixo Amazonas que abrange 2 UF's, deve-se colocar a condição de agregar, para este caso, as duas UF's 
# teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

In [0]:
# teste.to_excel('teste2.xlsx')
lista1.to_excel('lista1.xlsx')

In [0]:
teste.shape

In [0]:
teste.head()

In [0]:
teste['CD_MUNICIPIO'].str.len()

In [0]:
teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

In [0]:
teste['CD_MUNICIPIO'].str.len()

In [0]:
# verificar se tem regiões (ANTERIOR que abrangem mais de uma UF:
# df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF']).size()
df_unicos = df_regioes.drop_duplicates(subset=['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'])


In [0]:
df_unicos.shape  #362, 8

In [0]:
# Agrupa para selecionar Região de Saúde no DataFrame. Depois aplica o o metodo duplicated, para selecionar os efetivamente repetidos
linhas_duplicadas = df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'], as_index=False).count()
linhas_duplicadas.shape # 358, 8

In [0]:
# Seleciona todas as linhas duplicadas baseadas em uma lista de colunas. O parametro keep é para exibir todos os repetidos e não o último ou primeiro
linhas_duplicadas = linhas_duplicadas[linhas_duplicadas.duplicated(subset=['NM_REGIAO_SAUDE_ANTERIOR'], keep=False)]
linhas_duplicadas.shape  #(17, 8)
linhas_duplicadas
# Análise: Observando o campo NM_REGIAO_SAUDE_ANTERIOR e SG_UF, observa-se que: As regiões repetidas e que são de UF's contíguas São: Baixo Amazonas (AM e PA).
# O restante não são contíguas.


In [0]:
# ================ Fim dos Testes =====================

In [0]:
# =========================================
# OPERAÇÕES DE CONCATENAÇÃO DAS LISTAS DE MUNICIPIOS COM O DATAFRAME ORIGINAL

In [0]:
df_regioes_concat1 = pd.merge(df_regioes, lista1, how = 'left', left_on=['NM_REGIAO_SAUDE_ATUAL','SG_UF'], right_on=['NM_REGIAO_SAUDE_ATUAL','SG_UF'])

In [0]:
df_regioes_concat1.shape

In [0]:
# Renomear coluna que foi gerada na juncao do passo anterior

In [0]:
# Renomear coluna que foi gerada na juncao do passo anterior
df_regioes_concat1.rename(columns={'CD_MUNICIPIO_y':'lista_regiao_atual'}, inplace=True)

In [0]:
df_regioes_concat1.shape

In [0]:
lista1.head()

In [0]:
# Rotina para agregar a regiao 'Baixo Amazonas' das UF's: AM e PA, por tratar-se de região interestadual. Caso único.
lista_aux1 = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']
lista_aux2 = lista2[lista2['NM_REGIAO_SAUDE_ANTERIOR']=='Baixo Amazonas']



In [0]:
lista_aux1.head()

In [0]:
lista_aux2.head()

In [0]:
# uso do drop para SG_UF  para facilitar o gropuby por NM_REGIAO_SAUDE_ATUAL
lista_aux1_new = lista_aux1.drop('SG_UF', axis=1)
lista_aux2_new = lista_aux2.drop('SG_UF', axis=1)

In [0]:
lista_aux1_fim = lista_aux1_new.groupby('NM_REGIAO_SAUDE_ATUAL')['CD_MUNICIPIO'].sum().reset_index()
lista_aux2_fim = lista_aux2_new.groupby('NM_REGIAO_SAUDE_ANTERIOR')['CD_MUNICIPIO'].sum().reset_index()

In [0]:
lista_aux1_fim.head()

In [0]:
lista_aux2_fim.head()

In [0]:
# Concatenação com a Regiao anterior, na otica da lista 2

In [0]:
df_regioes_concat2 = pd.merge(df_regioes, lista2, how = 'left', left_on=['NM_REGIAO_SAUDE_ANTERIOR','SG_UF'], right_on=['NM_REGIAO_SAUDE_ANTERIOR','SG_UF'])

In [0]:
df_regioes_concat2.head()

In [0]:
df_regioes_concat2.rename(columns={'CD_MUNICIPIO_y':'lista_regiao_anterior'}, inplace=True)

In [0]:
# como são dois dataframes muito identicos em colunas, usei esta técnica para selecionar as do segundo dataframe que sejam diferentes
colunas_mantidas = df_regioes_concat2.columns.difference(df_regioes_concat1.columns)

In [0]:
colunas_mantidas

In [0]:
df_regioes_concat = pd.merge(df_regioes_concat1, df_regioes_concat2[colunas_mantidas], left_index=True, right_index=True, how='outer')

In [0]:
df_regioes_concat.head()

In [0]:
df_regioes_concat.rename(columns={'CD_MUNICIPIO_x':'CD_MUNICIPIO'}, inplace=True)

In [0]:
df_regioes_concat['flag_atual'] = np.where(df_regioes_concat['lista_regiao_atual'] == df_regioes_concat['lista_regiao_anterior'], 0, 1)

In [0]:
df_municipios = pd.read_excel('AR_BR_RG_UF_MES_MIC_MUN_2018.xls')

In [0]:
df_municipios.head()

In [0]:
df_municipios.dropna(axis=0, how='any', subset=['CD_GCMUN'], inplace=True)

In [0]:
df_municipios['CD_MUNICIPIO_NDV'] = df_municipios['CD_GCMUN'].astype(str).str[:6]

In [0]:
type(df_municipios['CD_MUNICIPIO_NDV'][0])

In [0]:
df_regioes_concat['CD_MUNICIPIO'] = df_regioes_concat['CD_MUNICIPIO'].astype(str)

In [0]:
df_diferenca = pd.merge(df_regioes_concat, df_municipios, how = 'outer', left_on='CD_MUNICIPIO', right_on='CD_MUNICIPIO_NDV')

In [0]:
df_diferenca_qgis = df_diferenca[df_diferenca['flag_atual']==1]

In [0]:
df_regioes_concat.rename(columns={'SG_UF_x':'SG_UF'}, inplace=True)

In [0]:
df_regioes_concat.to_excel('teste.xlsx')

In [0]:
df_regioes_concat.head()

In [0]:
resumo1 = df_regioes_concat[['SG_UF', 'NM_MUNICIPIO', 'NM_REGIAO_SAUDE_ATUAL']]

In [0]:
df_reg_mesmos_municipios = df_regioes_concat[df_regioes_concat['flag_atual']==0]

In [0]:
df_reg_mesmos_municipios.shape

In [0]:
municipios_sem_mudar_regiao = df_reg_mesmos_municipios.shape[0]

In [0]:
municipios_sem_mudar_regiao

In [0]:
df_reg_mesmos_municipios.groupby('NM_REGIAO_SAUDE_ATUAL').count()

In [0]:
df_reg_mesmos_municipios.groupby('NM_REGIAO_SAUDE_ANTERIOR').count()

In [0]:
resumo2 = df_reg_mesmos_municipios.groupby(['NM_REGIAO_SAUDE_ATUAL','NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF']).count()['CD_MUNICIPIO'].reset_index()

In [0]:
resumo2.sort_values('CD_MUNICIPIO', ascending=False, inplace=True)

In [0]:
resumo2

In [0]:
df_regioes_concat.sort_values('SG_UF', ascending=True, inplace=True)

In [0]:
for row in df_regioes_concat.loc[df_regioes_concat['lista_regiao_anterior'].isnull(), 'lista_regiao_anterior'].index:
    df_regioes_concat.at[row, 'lista_regiao_anterior'] = []

In [0]:
df_regioes_concat[df_regioes_concat['lista_regiao_anterior'].isnull()]

In [0]:
df_regioes_concat['flag_municipio_entrou'] = df_regioes_concat['lista_regiao_atual'].map(set) - df_regioes_concat['lista_regiao_anterior'].map(set)

In [0]:
df_regioes_concat['flag_municipio_saiu'] = df_regioes_concat['lista_regiao_anterior'].map(set) - df_regioes_concat['lista_regiao_atual'].map(set)

In [0]:
df_regioes_concat[df_regioes_concat['NM_REGIAO_SAUDE_ATUAL']=='Baixo Acre e Purus']

In [0]:
df_regioes_concat[df_regioes_concat['NM_REGIAO_SAUDE_ATUAL']=='Norte']

In [0]:
df_regioes_concat['cont_municipio_entrou'] = [len(x) for x in df_regioes_concat['flag_municipio_entrou']]

In [0]:
df_regioes_concat['cont_municipio_saiu'] = [len(x) for x in df_regioes_concat['flag_municipio_saiu']]

In [0]:
df_regioes_concat[(df_regioes_concat['flag_atual']==0)&(df_regioes_concat['cont_municipio_entrou']>1)]

In [0]:
df_regioes_concat[(df_regioes_concat['flag_atual']==0)&(df_regioes_concat['cont_municipio_saiu']>1)]

In [0]:
df_estatistica_entrada = df_regioes_concat[(df_regioes_concat['flag_atual']==1)&(df_regioes_concat['cont_municipio_entrou']!=0)]

In [0]:
df_estatistica_entrada.shape

In [0]:
df_result_atual = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ATUAL'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()

In [0]:
df_result_atual

In [0]:
df_result_anterior = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ANTERIOR'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()

In [0]:
df_result_anterior.head()

In [0]:
df_result_atual.to_excel('resultado_atual.xlsx')

In [0]:
df_result_anterior.to_excel('resultado_anterior.xlsx')

In [0]:
df_estatistica_entrada.to_excel('resumo_geral_movimentacao.xlsx')

In [0]:
# Sum list of values within a pandas df
# df = [sum(a) for a in zip(*df['Val'])]